In [1]:
# import library
import pybamm
import numpy as np

In [4]:
# define class
class MyThermal(pybamm.BaseModel):
    def __init__(self):
        super().__init__()

        # define variable (i.e. concentration in two electrodes)
        T = pybamm.Variable("Cell temperature", domain="full cell")

        # define parameters
        Heat_gen = pybamm.Parameter("Heat generation [W]")
        R = pybamm.Parameter("Cell radius [m]")
        k = pybamm.Parameter("Heat conductivity [W/m2-K]")
        h = pybamm.Parameter("Convection heat transfer coefficient [W/m2-K]")
        T0 = pybamm.Parameter("Intial temperature [K]")
        Height = pybamm.Parameter("Cell height [m]")
        A = (
            2
            * 3.14
            * pybamm.Parameter("Cell radius [m]")
            * pybamm.Parameter("Cell height [m]")
        )
        V = (
            3.14
            * (pybamm.Parameter("Cell radius [m]") ** 2)
            * pybamm.Parameter("Cell height [m]")
        )
        Tamb = pybamm.Parameter("Ambient temperature [K]")
        m = pybamm.Parameter("mass [kg]")
        ro = (
            m / V
        )  # pybamm.Parameter("mass [kg]")/ (3.14 * (pybamm.Parameter("Cell radius [m]")**2) * pybamm.Parameter("Cell height [m]"))
        Cp = pybamm.Parameter("Heat Capacity of the cell")

        # governing equations
        dTdt = (1 / (ro * Cp)) * pybamm.div(k * pybamm.grad(T)) + Heat_gen / (
            V * ro * Cp
        )
        self.rhs = {T: dTdt}

        # define intermediate variables and OCP function parameters
        T_s = pybamm.surf(T)

        # boundary conditions
        lbc = pybamm.Scalar(0)
        rbc = -h * (T_s - Tamb) / k
        self.boundary_conditions = {
            T: {"left": (lbc, "Neumann"), "right": (rbc, "Neumann")}
        }

        # initial condtions
        self.initial_conditions = {T: T0}

        T_s = pybamm.surf(T)
        # define output variables
        self.variables = {
            "Time [s]": pybamm.t,
            "Heat generation [W]": Heat_gen,
            "Surface temperature [K]": T_s,
        }

In [6]:
model = MyThermal()

R = pybamm.Parameter("Cell radius [m]")
# define geometry
r = pybamm.SpatialVariable("r", domain=["full cell"], coord_sys="cylindrical")
geometry = {"full cell": {r: {"min": pybamm.Scalar(0), "max": R}}}
# mesh and discretise
submesh_types = {"full cell": pybamm.Uniform1DSubMesh}
var_pts = {r: 50}

param = pybamm.ParameterValues(
    {
        "Heat generation [W]": 2,
        "Heat conductivity [W/m2-K]": 0.2,
        "Convection heat transfer coefficient [W/m2-K]": 40,
        "Intial temperature [K]": 300,
        "Cell height [m]": 0.07,
        "Cell radius [m]": 0.01,
        "Ambient temperature [K]": 300,
        "Heat Capacity of the cell": 1200,
        "mass [kg]": 0.07,
    }
)

param.process_geometry(geometry)

mesh = pybamm.Mesh(geometry, submesh_types, var_pts)
spatial_methods = {"full cell": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)

param.process_model(model)
disc.process_model(model)

In [7]:
solver = pybamm.ScipySolver()
t_eval = np.linspace(0, 7200, 100)
solution = solver.solve(model, t_eval)

In [8]:
solution.plot(["Surface temperature [K]", "Heat generation [W]", "Cell temperature"])

interactive(children=(FloatSlider(value=0.0, description='t', max=2.0, step=0.02), Output()), _dom_classes=('w…